In [2]:
import env 
import pandas as pd
import acquire as a
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")


In [2]:
# This will be part of my aquire file

#import the libraries we are working with 
import pandas as pd 
import os 

# we are using this fucntion to call the code needed to get a connection 
# between SQL and Python using my own ENV file where I keep my credentials
# Additionally, important to note that I hard coded the database in my env file and
# this was done on purpose to keep access to only one database. 

def get_connection(user=env.username,password=env.password,host=env.host,db=env.db):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'
    
query = '\
select * \
from customers\
join internet_service_types using(internet_service_type_id)\
join contract_types using(contract_type_id)\
join payment_types using(payment_type_id)\
join customer_signups using (customer_id)\
join customer_churn using(customer_id)\
join customer_contracts using (customer_id)\
join customer_details using (customer_id)\
join customer_payments using (customer_id)\
join customer_subscriptions using (customer_id)'

def get_telco_data():
    #if os.path.exists('telco.csv') == True:
       # return pd.read_csv('telco.csv')
   # else:
       return pd.read_sql(query, get_connection())
       # df.to_csv('telco.csv')
        # return pd.read_csv('telco.csv')

In [3]:
df = a.get_telco_data()

# **Prepare:**

### **Summarizing our Data:** 

# First we want to check what columnns we want to get rid of. 
# We'll do this by checking the DataFrame

> While preparing my data, i notice that i prodomiantly used the _id columns to link more viable 
> columns so im going to make a list of all the columns that 
> contain the _id to verify we can get rid of them. 

In [8]:
# id_list = []
# for cols in df.columns:
#     if '_id' in cols:
#         id_list.append(cols)
#         df =df.drop(columns=id_list)
#     else:
#         pass
id_list

['customer_id', 'payment_type_id']

# After Looking at my data I noticed unnamed: 0 is not a relevant field I want to keep

In [81]:
id_list.append('Unnamed: 0')

# Here I trusted but veridfied that the these values were of no value and had a readable
# Columns equivalient now i can remove the columns.

In [82]:
for cols in df.columns:
    if '.1' in cols:
        id_list.append(cols)
#df

# After looking at my data I can see NaN in my churn month so im gonna investigate 
# further 
# we tested to see how many nan there is in by converting the column intop a bool
# value where if it is true it will be repersented by a 1. If we take the sum of all
# the true bool values (the values that nan) and divide by the length of the column we get 
# that aprox. 0.73 of our values are nan so this isnt a reliable column and can be droped 


In [83]:
(df['churn_month'].isnull().sum()) / (len(df['churn_month']))
id_list.append('churn_month')
drop_cols = id_list
df = df.drop(columns=drop_cols)

In [84]:
# recall data frame and add any unwanted/un-needed columns to my_list
#columns count is still higher than id like im gonna do another look to see if there is
# anything I can ditch 
df = df.drop(columns='signup_date')
# lets plopt some to see if we can see any visual reason to keep the columns 


# Here we can see the percentage of customers who dont have internet service in their plan 

In [85]:
round(((df == 'No internet service').sum()) / (len(df)) * 100,2)

gender                    0.00
senior_citizen            0.00
partner                   0.00
dependents                0.00
tenure                    0.00
phone_service             0.00
multiple_lines            0.00
online_security          21.67
online_backup            21.67
device_protection        21.67
tech_support             21.67
streaming_tv             21.67
streaming_movies         21.67
paperless_billing         0.00
monthly_charges           0.00
total_charges             0.00
churn                     0.00
internet_service_type     0.00
contract_type             0.00
payment_type              0.00
dtype: float64

# We are looking at 21.666903 % (or 21.67 when rounded) of customers dont have internet service

# Since these can be converted to bool values. I dont want to drop these specific columns as there may be a connection between customers who use some of these features and the probaility of churning

# Since the customers dont have internet service so by default dont uses these  features im going to change the values to no and change the yes and no to bianry 

In [86]:
df = df.replace('No internet service', 'no')


In [87]:
# now we change the Nos and Yes to the purest form of True and False, which is 1's & 0's
# we also do is the same with gender and so we can be clear,
# on which sex is which when converting to bianary, 
change_values = {
    'No phone service' : 0,
    'Male': 1,
    'Female': 0,
    'no' : 0,
    'No' : 0,
    'Yes' : 1
    
}

df = df.rename(columns = {'gender': 'sex_male'})
df = df.replace(change_values)



# Now, lets see how this dataframe is starting to look

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sex_male               7043 non-null   int64  
 1   senior_citizen         7043 non-null   int64  
 2   partner                7043 non-null   int64  
 3   dependents             7043 non-null   int64  
 4   tenure                 7043 non-null   int64  
 5   phone_service          7043 non-null   int64  
 6   multiple_lines         7043 non-null   int64  
 7   online_security        7043 non-null   int64  
 8   online_backup          7043 non-null   int64  
 9   device_protection      7043 non-null   int64  
 10  tech_support           7043 non-null   int64  
 11  streaming_tv           7043 non-null   int64  
 12  streaming_movies       7043 non-null   int64  
 13  paperless_billing      7043 non-null   int64  
 14  monthly_charges        7043 non-null   float64
 15  tota

In [89]:
# This will eventually become a function
my_object_list = []
for cols in df.columns:
    if df[cols].dtype == 'O':
        my_object_list.append(cols)
    else:
       pass

# Now lets check to see what these
# object columns to see if we can
# standardize them. 

In [90]:
my_object_list

['total_charges', 'internet_service_type', 'contract_type', 'payment_type']

# Lets Start with total charges 

# It looks like it is missing a zero to make it a non object type cell so lets fix that 


In [98]:
df.

,sex_male,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,...,internet_service_type_dsl,internet_service_type_fiber_optic,internet_service_type_none,contract_type_month-to-month,contract_type_one_year,contract_type_two_year,payment_type_bank_transfer_(automatic),payment_type_credit_card_(automatic),payment_type_electronic_check,payment_type_mailed_check
1292,1,1,0,0,55,0,0,0,0,1,...,1,0,0,0,1,0,1,0,0,0
1278,0,0,0,0,39,1,0,0,1,0,...,1,0,0,0,1,0,0,1,0,0
6132,0,0,1,1,23,1,0,1,1,0,...,0,1,0,1,0,0,0,0,0,1
336,1,1,1,0,5,1,1,0,0,1,...,0,1,0,1,0,0,1,0,0,0
6760,0,0,1,1,35,1,0,1,0,1,...,0,1,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2868,1,0,1,1,34,1,1,0,1,0,...,0,1,0,1,0,0,1,0,0,0
6445,0,0,1,1,69,1,1,0,1,1,...,0,1,0,0,1,0,1,0,0,0
1291,0,0,0,0,5,1,0,0,0,0,...,0,0,1,1,0,0,0,1,0,0
2730,1,0,0,0,22,1,0,0,1,0,...,1,0,0,0,1,0,0,1,0,0


In [97]:
df.total_charges = df['total_charges'].replace(' ', 0)
df.total_charges = df['total_charges'].astype('float64')

KeyError: 'total_charges'

# Now lets run the for loop again to see if it worked.

In [93]:
my_object_list = []
for cols in df.columns:
    if df[cols].dtype == 'O':
        my_object_list.append(cols)
    else:
       pass
my_object_list
# No more total_charges 

['internet_service_type', 'contract_type', 'payment_type']

In [94]:
df.dtypes

sex_male                   int64
senior_citizen             int64
partner                    int64
dependents                 int64
tenure                     int64
phone_service              int64
multiple_lines             int64
online_security            int64
online_backup              int64
device_protection          int64
tech_support               int64
streaming_tv               int64
streaming_movies           int64
paperless_billing          int64
monthly_charges          float64
total_charges            float64
churn                      int64
internet_service_type     object
contract_type             object
payment_type              object
dtype: object

# Now we can move forward with the other objects.

# Looks like we can actually split all the values  and get dummy values for these object columns. 


In [95]:
get_dummy_vales = (pd.get_dummies(df[my_object_list]))
df = pd.concat([df, get_dummy_vales], axis=1)

# Now I can drop the object columns


In [96]:
my_object_list = []
for cols in df.columns:
    if df[cols].dtype == 'O':
        my_object_list.append(cols)
    else:
       pass

In [97]:
pd.set_option('max_colwidth', None)    
df.columns = (df.columns.str).lower().str.replace(' ', '_')

In [98]:
import prepare as p
prepped_df = df.drop(columns=my_object_list)

In [5]:
import acquire as a
import prepare as p
df = a.get_telco_data()
prepped_df = p.prepare_telco(df)
prepped_df = prepped_df.drop(columns = ['payment_type_id', 'contract_type_id', 'internet_service_type_id'])

In [44]:
train.columns = train.columns.str.replace(' ','_').str.lower()

In [10]:
now_order_drop =['sex_male', 'senior_citizen', 'partner', 'dependents', 'tenure',
       'phone_service', 'multiple_lines', 'online_security', 'online_backup',
       'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
       'paperless_billing', 'monthly_charges','internet_service_type_DSL', 'internet_service_type_Fiber optic',
       'internet_service_type_None', 'contract_type_Month-to-month',
       'contract_type_One year', 'contract_type_Two year',
       'payment_type_Bank transfer (automatic)',
       'payment_type_Credit card (automatic)', 'payment_type_Electronic check',
       'payment_type_Mailed check','churn',]
prepped_df.columns = now_order_drop

In [45]:
prepped_df
prepped_df = prepped_df.loc[~prepped_df.duplicated()]

# somewhere in the prepare we are losing total charges
# lets invesetigate

In [99]:
# reset df everytime we run this block:
df = a.get_telco_data()

# were gonna berak it down block by block 

# block one:
id_list = []

for cols in df.columns:
    if '.1' in cols:
        id_list.append(cols)
    else:
         pass

# total charges are not in this list my hunch is that it is in the object section.

# block two:
# in the next few block we are generatring a list of columns we dont want so we can drop them at the end  
for cols in df.columns:       
        if '_id' in cols:
            id_list.append(cols)
        else:
            pass



# block two is also not the issue, as ttotal charges also dont appear in this list 

#block three: 

# Here we are adding more columns to the list of col¨mns to be dropped. 
id_list.append('Unnamed: 0')
(df['churn_month'].isnull().sum()) / (len(df['churn_month']))
id_list.append('churn_month')
id_list.append('customer_id')
id_list.append('signup_date')
change_values = {
    'No phone service' : 0,
    'Male': 1,
    'Female': 0,
    'no' : 0,
    'No' : 0,
    'Yes' : 1,
    'No internet service': 'no'
}
df.columns
#still good here 
df = df.rename(columns = {'gender': 'sex_male'})
df = df.replace(change_values)
df.total_charges = df['total_charges'].replace(' ', 0)
df.total_charges = df['total_charges'].astype('float64')
for cols in df.columns:
    if df[cols].dtype == 'O':
        id_list.append(cols)
    else:
       pass

df = df.drop(columns=id_list)
id_list


NameError: name 'id_list_one' is not defined

In [46]:
def split_data(prepped_df):
    train_validate, test = train_test_split(prepped_df, test_size=.2, random_state=123, stratify=prepped_df.churn)
    train, validate = train_test_split(train_validate, 
                                       test_size=.3, 
                                       random_state=123, 
                                       stratify=train_validate.churn)
    return train, validate, test


In [47]:
train, validate, test = split_data(prepped_df)
train

,sex_male,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,...,internet_service_type_DSL,internet_service_type_Fiber optic,internet_service_type_None,contract_type_Month-to-month,contract_type_One year,contract_type_Two year,payment_type_Bank transfer (automatic),payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
1292,1,1,0,0,55,0,0,0,0,1,...,1,0,0,0,1,0,1,0,0,0
1278,0,0,0,0,39,1,0,0,1,0,...,1,0,0,0,1,0,0,1,0,0
6132,0,0,1,1,23,1,0,1,1,0,...,0,1,0,1,0,0,0,0,0,1
336,1,1,1,0,5,1,1,0,0,1,...,0,1,0,1,0,0,1,0,0,0
6760,0,0,1,1,35,1,0,1,0,1,...,0,1,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2868,1,0,1,1,34,1,1,0,1,0,...,0,1,0,1,0,0,1,0,0,0
6445,0,0,1,1,69,1,1,0,1,1,...,0,1,0,0,1,0,1,0,0,0
1291,0,0,0,0,5,1,0,0,0,0,...,0,0,1,1,0,0,0,1,0,0
2730,1,0,0,0,22,1,0,0,1,0,...,1,0,0,0,1,0,0,1,0,0


## Now lets explore 

In [60]:
df = train
train.head()

,sex_male,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,...,internet_service_type_dsl,internet_service_type_fiber_optic,internet_service_type_none,contract_type_month-to-month,contract_type_one_year,contract_type_two_year,payment_type_bank_transfer_(automatic),payment_type_credit_card_(automatic),payment_type_electronic_check,payment_type_mailed_check
1292,1,1,0,0,55,0,0,0,0,1,...,1,0,0,0,1,0,1,0,0,0
1278,0,0,0,0,39,1,0,0,1,0,...,1,0,0,0,1,0,0,1,0,0
6132,0,0,1,1,23,1,0,1,1,0,...,0,1,0,1,0,0,0,0,0,1
336,1,1,1,0,5,1,1,0,0,1,...,0,1,0,1,0,0,1,0,0,0
6760,0,0,1,1,35,1,0,1,0,1,...,0,1,0,1,0,0,0,0,1,0


In [53]:
train.columns = train.columns.str.lower().str.replace(' ', '_')

In [66]:
train.columns

Index(['sex_male', 'senior_citizen', 'partner', 'dependents', 'tenure',
       'phone_service', 'multiple_lines', 'online_security', 'online_backup',
       'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
       'paperless_billing', 'monthly_charges', 'churn',
       'internet_service_type_dsl', 'internet_service_type_fiber_optic',
       'internet_service_type_none', 'contract_type_month-to-month',
       'contract_type_one_year', 'contract_type_two_year',
       'payment_type_bank_transfer_(automatic)',
       'payment_type_credit_card_(automatic)', 'payment_type_electronic_check',
       'payment_type_mailed_check'],
      dtype='object')

# Let's check and see if we missed any columns the can be removed

In [96]:
train.columns
# Lets break the columns down into categories to see if we can see correlations 

demographics = ['sex_male', 'senior_citizen', 'partner', 'dependents']

payment_types = ['payment_type_bank_transfer_(automatic)','payment_type_credit_card_(automatic)', 'payment_type_electronic_check',\
                   'payment_type_mailed_check']
services_provided = ['phone_service', 'multiple_lines', 'online_security', 'online_backup',
       'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',]
contract_types = ['contract_type_month-to-month',
       'contract_type_one_year', 'contract_type_two_year']
internet_type = ['internet_service_type_dsl', 'internet_service_type_fiber_optic',
       'internet_service_type_none']

billing = ['paperless_billing', 'monthly_charges', 'tenure']

churn = ['churn']





df.columns


Index(['sex_male', 'senior_citizen', 'partner', 'dependents', 'tenure',
       'phone_service', 'multiple_lines', 'online_security', 'online_backup',
       'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
       'paperless_billing', 'monthly_charges', 'churn',
       'internet_service_type_dsl', 'internet_service_type_fiber_optic',
       'internet_service_type_none', 'contract_type_month-to-month',
       'contract_type_one_year', 'contract_type_two_year',
       'payment_type_bank_transfer_(automatic)',
       'payment_type_credit_card_(automatic)', 'payment_type_electronic_check',
       'payment_type_mailed_check'],
      dtype='object')